In [1]:
from get_server_data import get_db_table_as_df, db_filename
import sqlite3
import pandas as pd
import numpy as np

In [2]:
flood_locations = get_db_table_as_df('flood_locations')
flood_events = get_db_table_as_df('flood_events', date_col=['event_date', 'dates'])
for_model = get_db_table_as_df('for_model', date_col=['event_date'])
daily_obs = get_db_table_as_df('dntwn_nor_daily_observations', date_col=['Datetime'])

In [3]:
flood_locations.set_index('location', inplace=True)
flood_locations.head()

,xcoord,ycoord,lon,lat,elevation,distance_to_OW,distance_to_OW_EW,distance_to_OW_WW,flood_count,is_downtown
location,,,,,,,,,,
300 BLOCK OF SHIRLEY AVENUE,1.212926e+07,3.481753e+06,-76.290425,36.864399,2.27047,3291.649848,3291.649848,3291.649848,5,1
LLEWELLYN AVENUE & SHIRLEY AVENUE,1.212972e+07,3.481566e+06,-76.288847,36.863856,2.15762,3305.283331,3305.283331,3305.283331,9,1
LLEWELLYN AVENUE & W PRINCESS ANNE ROAD,1.212937e+07,3.480689e+06,-76.290115,36.861472,2.31289,2368.435322,2368.435322,2368.435322,1,1
900 BLOCK OF TIDEWATER DRIVE,1.213392e+07,3.478090e+06,-76.274775,36.854041,1.73326,4374.592858,4456.503365,3139.348615,1,1
CAMPOSTELLA ROAD & E BRAMBLETON AVENUE,1.213638e+07,3.474443e+06,-76.266674,36.843867,3.46408,714.107730,540.656507,759.373034,2,1


In [4]:
daily_obs.set_index('Datetime', inplace=True)
daily_obs.head()

,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,rain_15_min_max,rain_15_min_max_time,rain_prev_3_days,gw_daily_avg,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg
Datetime,,,,,,,,,,,,,
2010-01-01,0.055,0.04,2010-01-01 22:45:00,0.02,2010-01-01 22:30:00,0.145,3.104508,0.387667,1.654,1.581,353.0,6.75,NaN
2010-01-02,0.000,0.00,None,0.00,None,0.130,3.414894,0.352125,NaN,NaN,321.0,16.90,NaN
2010-01-03,0.000,0.00,None,0.00,None,0.055,3.120831,-0.853333,NaN,NaN,295.0,13.90,NaN
2010-01-04,0.050,0.10,2010-01-04 13:30:00,0.10,2010-01-04 13:30:00,0.050,2.948871,-0.789292,0.279,0.279,306.0,8.50,NaN
2010-01-05,0.000,0.00,None,0.00,None,0.050,2.833045,-0.235708,NaN,NaN,292.0,7.25,NaN


In [5]:
flood_events.head()

,location,eventType,event_name,event_date,dates,times
0,300 BLOCK OF SHIRLEY AVENUE,Flooded street,Nicole,2010-09-30,2010-09-30,07:32:47.000
1,LLEWELLYN AVENUE & SHIRLEY AVENUE,Flooded street,Nicole,2010-09-30,2010-09-30,07:33:19.000
2,LLEWELLYN AVENUE & W PRINCESS ANNE ROAD,Flooded street,Nicole,2010-09-30,2010-09-30,07:33:48.000
3,900 BLOCK OF TIDEWATER DRIVE,Flooded street,Nicole,2010-09-30,2010-09-30,08:29:00.000
4,CAMPOSTELLA ROAD & E BRAMBLETON AVENUE,Flooded street,Nicole,2010-09-30,2010-09-30,08:29:38.000


In [6]:
for_model.set_index('event_date', inplace=True)
for_model.head()

,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,rain_15_min_max,rain_15_min_max_time,rain_prev_3_days,gw_daily_avg,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg,num_flooded,flooded,event_name
event_date,,,,,,,,,,,,,,,,
2010-09-30,11.415000,1.59,2010-09-30 04:15:00,0.67,2010-09-30 15:45:00,11.695000,3.051157,1.111472,0.961,0.577,291.666667,13.200000,NaN,48.0,1,Nicole
2011-08-27,7.895000,1.22,2011-08-27 11:30:00,0.33,2011-08-27 11:00:00,8.245000,1.806668,1.493500,2.602,2.516,155.500000,31.000000,NaN,32.0,1,Irene
2012-10-28,3.795000,0.58,2012-10-28 22:15:00,0.21,2012-10-28 21:45:00,2.495000,2.647606,2.903833,4.245,4.245,148.858198,23.504701,22.147917,45.0,1,Sandy
2013-10-09,4.320000,0.50,2013-10-09 15:00:00,0.22,2013-10-10 01:15:00,3.825000,1.661222,1.949806,3.839,2.959,39.311475,19.630806,18.424167,6.0,1,Heavy Rain
2014-05-16,3.853333,2.23,2014-05-16 08:00:00,0.76,2014-05-16 07:30:00,3.883333,2.208672,0.328958,0.643,0.643,214.833146,6.531076,8.279167,21.0,1,Heavy Rain


In [7]:
dwntwn_locations = flood_locations[flood_locations['is_downtown']==1].index

In [8]:
lst = []
for l in dwntwn_locations:
    event_dates = flood_events[flood_events['location'] == l]['event_date']
    all_fld_dates = flood_events[flood_events['event_date'].isin(event_dates)]['dates']
    all_fld_dates = np.append(all_fld_dates, event_dates)
    all_fld_dates = pd.to_datetime(np.unique(all_fld_dates))

    fld_data = for_model[for_model.index.isin(event_dates)]
    
    loc_data = daily_obs.copy()
    loc_data = loc_data[~loc_data.index.isin(all_fld_dates)]
    loc_w_fld = pd.concat([loc_data, fld_data])
    
    # add geog data
    loc_w_fld['location'] = l
    geog_data = flood_locations.loc[l, :]
    for k in geog_data.keys():
        loc_w_fld[k] = geog_data[k]
    lst.append(loc_w_fld)
        

In [9]:
all_locations = pd.concat(lst)
all_locations.reset_index(inplace=True)

In [10]:
new_cols = all_locations.columns.tolist()
new_cols[0] = 'Date'
all_locations.columns = new_cols
all_locations.head()

,Date,event_name,flooded,gw_daily_avg,num_flooded,rain_15_min_max,rain_15_min_max_time,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,...,xcoord,ycoord,lon,lat,elevation,distance_to_OW,distance_to_OW_EW,distance_to_OW_WW,flood_count,is_downtown
0,2010-01-01,NaN,NaN,3.104508,NaN,0.02,2010-01-01 22:30:00,0.055,0.04,2010-01-01 22:45:00,...,1.212926e+07,3.481753e+06,-76.290425,36.864399,2.27047,3291.649848,3291.649848,3291.649848,5.0,1.0
1,2010-01-02,NaN,NaN,3.414894,NaN,0.00,None,0.000,0.00,None,...,1.212926e+07,3.481753e+06,-76.290425,36.864399,2.27047,3291.649848,3291.649848,3291.649848,5.0,1.0
2,2010-01-03,NaN,NaN,3.120831,NaN,0.00,None,0.000,0.00,None,...,1.212926e+07,3.481753e+06,-76.290425,36.864399,2.27047,3291.649848,3291.649848,3291.649848,5.0,1.0
3,2010-01-04,NaN,NaN,2.948871,NaN,0.10,2010-01-04 13:30:00,0.050,0.10,2010-01-04 13:30:00,...,1.212926e+07,3.481753e+06,-76.290425,36.864399,2.27047,3291.649848,3291.649848,3291.649848,5.0,1.0
4,2010-01-05,NaN,NaN,2.833045,NaN,0.00,None,0.000,0.00,None,...,1.212926e+07,3.481753e+06,-76.290425,36.864399,2.27047,3291.649848,3291.649848,3291.649848,5.0,1.0


In [11]:
con = sqlite3.connect(db_filename)
all_locations.to_sql(con=con, name='for_model_geog', if_exists='replace')